In [1]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import cv2
import numpy as np

### Video postprocesseing

In [2]:
import os
import pickle
import numpy as np
import cv2 as cv

class VideoPreprocessor:
    def __init__(self, directory, video_path):
        self.directory = directory
        self.video_path = video_path
        self.fps = None
        self.frame_count = 0
        self.extracted_frame_count = 0

    def empty_folder(self):
        for file in os.listdir(self.directory):
            file_path = os.path.join(self.directory, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        return True

    def extract_frames_per_second(self):
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                raise IOError("Error opening video!")

            self.fps = cap.get(cv.CAP_PROP_FPS)
            if self.fps <= 0:
                raise ValueError("Invalid FPS value.")

            if self.empty_folder():
                while True:
                    ret, frame = cap.read()

                    if not ret:
                        print("End of video reached.")
                        break

                    # Extract one frame per second
                    if self.frame_count % int(self.fps) == 0:
                        filename = f"{self.directory}/frame_{self.extracted_frame_count}.jpg"
                        cv.imwrite(filename, frame)
                        self.extracted_frame_count += 1

                    self.frame_count += 1

                print(f"Extracted {self.extracted_frame_count} frames to {self.directory}")

        except IOError as e:
            print(f"IOError: {e}")
        except ValueError as e:
            print(f"ValueError: {e}")
        finally:
            cap.release()

# Example usage
if __name__ == "__main__":
    directory = "Frames_spd"
    video_path = "/Users/naveenpoliasetty/Downloads/David2.0/david2.0/WhatsApp Video 2024-08-08 at 22.19.13.mp4"

    if not os.path.exists(directory):
        os.makedirs(directory)

    preprocessor = VideoPreprocessor(directory, video_path)
    preprocessor.extract_frames_per_second()

End of video reached.
Extracted 8 frames to Frames_spd


## Detecting faces using retina face

In [3]:
from retinaface import RetinaFace

resp = RetinaFace.detect_faces("/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames/frame_1.jpg")

In [ ]:
x,y, w, h =  resp['face_1']['facial_area']
x2, y2, w2, h2 = resp['face_2']['facial_area']
x3, y3, w3, h3 = resp['face_3']['facial_area']
x4, y4, w4, h4 = resp['face_4']['facial_area']

In [ ]:
image = cv2.imread('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames/frame_1.jpg')
cv2.rectangle(image, (x, y), (w, h), (255, 0, 0), 2,)
plt.imshow(image)

In [ ]:
faces = RetinaFace.extract_faces(img_path = "/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames/frame_1.jpg", align = True)
for index, face in enumerate(faces):
  plt.imshow(face)
  plt.show()

## Using Face recognition module

In [ ]:
import face_recognition

# Load an image file into a numpy array
image = face_recognition.load_image_file("/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames/frame_1.jpg")

# Find all face locations in the image
face_locations = face_recognition.face_locations(image,model='cnn')

# Generate face embeddings for each detected face
face_encodings = face_recognition.face_encodings(image, face_locations)

# face_locations and face_encodings are lists of tuples


In [4]:
def cnn_model(path):
    image = face_recognition.load_image_file(path)
    face_locations =  face_recognition.face_locations(image, model='cnn')
    face_encodings = face_recognition.face_encodings(image, face_locations)
    return face_encodings

In [ ]:
import itertools

values = [557, 566, 639, 484]

# Generate all permutations
permutations = list(itertools.permutations(values))

# Convert the tuples into lists if needed
permutations = [list(p) for p in permutations]

# Print the permutations
for perm in permutations:
    cv2.rectangle(image, (perm[0], perm[1]),(perm[2], perm[3]), (255,0,0), thickness=3,)
    plt.imshow(image)
    plt.show()


In [ ]:
plt.imshow(image)

In [ ]:
x, y, w, h = 593, 282, 701, 175

cv2.rectangle(image, (x, y),(w, h), (255,0,0), thickness=3)
plt.imshow(image)

## Algos

In [155]:
def face_detections_retina_locations(path):
    locations = []
    image = cv2.imread(path)
    res = RetinaFace.detect_faces(path)
    for x in res.values():
        #xi, y, w, h = x['facial_area']
        locations.append(tuple(x['facial_area']))
    face_embeddings = face_recognition.face_encodings(image, locations)

    '''for x , y, w, h in locations:
        cv2.rectangle(image, (x, y), (w, h), (255, 0, 0), 2,)
        plt.imshow(image)
        plt.show()'''
    
    return face_embeddings

def cnn_model(path):
    image = face_recognition.load_image_file(path)
    face_locations =  face_recognition.face_locations(image, model='cnn')
    face_encodings = face_recognition.face_encodings(image, face_locations)
    return face_encodings

def face_detections_vggface(path):
    embedding_objs = DeepFace.represent(
    img_path = path,
    model_name='VGG-Face',
    )
    face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
    return face_embeddings

def face_detections_facenet(path):
    embedding_objs = DeepFace.represent(model_name='Facenet',
    img_path = path,
    )
    face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
    return face_embeddings

def face_detections_deepface(path):
    embedding_objs = DeepFace.represent(model_name='DeepFace',
        img_path = path,
        )
    face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
    return face_embeddings

In [128]:
import cv2
from deepface import DeepFace
import face_recognition
import logging

# Configure logging to capture errors
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

def face_detections_retina_locations(path):
    try:
        locations = []
        image = cv2.imread(path)
        if image is None:
            raise ValueError(f"Image at {path} could not be loaded.")
        res = RetinaFace.detect_faces(path)
        for x in res.values():
            locations.append(tuple(x['facial_area']))
        face_embeddings = face_recognition.face_encodings(image, locations)
        if len(face_embeddings) == 0:
                    pass
        return face_embeddings
    except Exception as e:
        logging.error(f"Error in face_detections_retina_locations with path {path}: {str(e)}")
        if len(face_embeddings) == 0:
            print('NO embeddings generated')
            pass
        return None

def cnn_model(path):
    try:
        image = face_recognition.load_image_file(path)
        face_locations = face_recognition.face_locations(image, model='cnn')
        face_encodings = face_recognition.face_encodings(image, face_locations)
        if len(face_encodings) == 0:
            print('NO embeddings generated')
            pass
        return face_encodings
    except Exception as e:
        logging.error(f"Error in cnn_model with path {path}: {str(e)}")
        if len(face_encodings) == 0:
            print('NO embeddings generated')
            pass
        return None

def face_detections_vggface(path):
    try:
        embedding_objs = DeepFace.represent(img_path=path, model_name='VGG-Face')
        face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
        if len(face_embeddings) == 0:
            print('NO embeddings generated')
            pass
        return face_embeddings
    except Exception as e:
        logging.error(f"Error in face_detections_vggface with path {path}: {str(e)}")
        if len(face_embeddings) == 0:
            print('NO embeddings generated')
            pass
        return None

def face_detections_facenet(path):
    try:
        embedding_objs = DeepFace.represent(model_name='Facenet', img_path=path)
        face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
        if len(face_embeddings) == 0:
            print('NO embeddings generated')
            return None
        return face_embeddings
    except Exception as e:
        logging.error(f"Error in face_detections_facenet with path {path}: {str(e)}")
        if len(face_embeddings) == 0:
            print('NO embeddings generated')
            pass
        return None

def face_detections_deepface(path):
    try:
        embedding_objs = DeepFace.represent(model_name='DeepFace', img_path=path)
        face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
        if len(face_embeddings) == 0:
            print('NO embeddings generated')
            pass
        return face_embeddings
    except Exception as e:
        logging.error(f"Error in face_detections_deepface with path {path}: {str(e)}")
        if len(face_encodings) == 0:
            print('NO embeddings generated')
            return None
        return None


In [112]:
paths = os.listdir('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/faces')

In [113]:
paths

['213J1A4288.jpg',
 '213J1A42C9.jpg',
 '213J1A42B9.jpg',
 '213J1A4298.jpg',
 '213J1A4267.jpg',
 '213J1A42A6.jpg',
 '223J5A4208.jpg',
 '223J5A4209.jpg',
 '213J1A42B6.jpg',
 '213J1A42C2.jpg',
 '213J1A4297.jpg',
 '213J1A42C3.jpg',
 '213J1A4280.jpg',
 '213J1A42B1.jpg',
 '213J1A4287.jpg',
 '213J1A42C6.jpg',
 '213J1A42B2.jpg',
 '223J5A42011.jpg']

In [156]:
paths = os.listdir('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/faces')
cnn_embeddings = []
facenet_embeddings = []
retinaface_embeddings = []
count = 0

for path in paths:
    embedding = cnn_model(f'faces/{path}')
    if embedding:
        cnn_embeddings.append(embedding)
    embedding = face_detections_retina_locations(f'faces/{path}')
    if embedding:
        retinaface_embeddings.append(embedding)
    embedding = face_detections_vggface(f'faces/{path}')
    if embedding:
        retinaface_embeddings.append(embedding)

    count += 1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [160]:
print(len(retinaface_embeddings))
print(len(cnn_embeddings))
print(len(facenet_embeddings))

35
18
0


In [151]:
def build_strings(array):
    new_list = []
    for nested in array:
         new_list.append("/".join([str(i) for i in nested[0]]))
    return new_list 
new_cnn_embedings = build_strings(cnn_embeddings)
new_face_net_embeddings= build_strings(facenet_embeddings)
new_retinaface_embeddings = build_strings(retinaface_embeddings)

In [154]:
print(len(new_retinaface_embeddings))

35


In [152]:
print(len(new_cnn_embedings))

18


In [153]:
print(len(facenet_embeddings))

0


In [117]:
print(len(facenet_embeddings))
print(facenet_embeddings[0][0])

18
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03226967865739602, 0.00719163669372287, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008144438374901137, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006008303232347897, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00250426732092617, 0.0, 0.0, 0.0, 0.0, 0.06240878661342931, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001439347877968591, 0.0, 0.0, 0.0, 0.0, 0.02471453494705524, 0.0903403455487666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007025454000293356, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015683020216728245, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009631067295712059, 0.0, 0.0, 0.0, 0.0, 0.04445085951062399, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
len(embeddings)

19

In [ ]:
frame_face_embeddings = face_detections_facenet('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames/frame_1.jpg')

In [ ]:
frame_face_embeddings

In [ ]:
real_face_locations, real_embeddings = face_detections_retina_locations('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/faces/213J1A4280.jpg')

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)

    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    cosine_sim = dot_product / (norm_vec1 * norm_vec2)
    
    return cosine_sim


In [ ]:
def compare_faces(embedd1, indiviual_faces_embeddings):
    scores, dot_scores = [], []
    for x in embedd1:
        scores.append(cosine_similarity(indiviual_faces_embeddings, x))
        dot_scores.append(np.dot(indiviual_faces_embeddings, x))
    return np.argmax(scores), np.argmax(dot_scores)


In [ ]:
compare_faces(frame_face_embeddings, real_embeddings)

In [ ]:
scores = []
dot_scores = []
for x in frame_face_embeddings:
    scores.append(cosine_similarity(real_embeddings, x))
    dot_scores.append(np.dot(real_embeddings, x))
print(np.argmax(scores))
print(np.argmax(dot_scores))

In [ ]:
dot_scores

In [ ]:
import numpy as np
print(np.dot(real_embeddings, frame_face_embeddings[0]))

# Edhi vere katha

In [ ]:
import os

path = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames_spd'
address = os.listdir(path)
address = ['/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames_spd/' + x for x in address]
faces = []

def generate_embeddings(address):
    locations = []
    meta_data = []
    for index, x in enumerate(address):
        res = DeepFace.extract_faces(x,enforce_detection=False)
        for i in range(len(res)):
            meta_data.append(res)
            locations.append((res[i]['facial_area']['x'] ,res[i]['facial_area']['y'] , (res[i]['facial_area']['w']+res[i]['facial_area']['x']) , (res[i]['facial_area']['h']+res[i]['facial_area']['y'])) )
    return locations, meta_data
    
embeddings, meta_data = generate_embeddings(address=address)


In [ ]:
address

In [ ]:
meta_data['array']

In [ ]:
from mtcnn import MTCNN
from PIL import Image
import numpy as np

# Initialize MTCNN
detector = MTCNN()

path = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames_spd'
address = os.listdir(path)
address = ['/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames_spd/' + x for x in address]

# Load the image

coordinates = []
for x in address:
    image = Image.open(x)
    image_np = np.array(image)

# Detect faces
    results = detector.detect_faces(image_np)

    # Print the coordinates of the detected faces
    for result in results:
        x, y, width, height = result['box']
        coordinates.append((x, y, x+width, y+height))
        

In [ ]:
coordinates

## Face mapping

In [ ]:
coordinates

In [ ]:
from PIL import Image

def extract_bbox(image_path, bbox):
    # Open the image file
    with Image.open(image_path) as img:
        left, upper, right, lower = bbox
        
        # Crop the image to the bounding box
        cropped_image = img.crop((left, upper, right, lower))
        
        return cropped_image

# Example usage
image_path = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/expirements/Frames/frame_0.jpg' # Example bounding box coordinates
patches = [f'/Users/naveenpoliasetty/Downloads/David2.0/david2.0/Frames_spd/frame_{x}.jpg' for x in range(len(os.listdir(path)))]

for patch in patches:
    for index, x in enumerate(coordinates):
        cropped_image = extract_bbox(patch, x)
        cropped_image.save(f'cropped_image{index}.jpg')  # Save the cropped image


In [ ]:
mappings = ['cropped_image' + str(x) + '.jpg' for x in range(7)]
students = os.listdir('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/SPD')
students = ['/Users/naveenpoliasetty/Downloads/David2.0/david2.0/SPD/' + s for s in students]
ans = []
sols = []
for x in mappings:
    for y in students:
        sol = DeepFace.verify(x,y,enforce_detection=False)
        if sol['verified'] == True :
            sols.append(sol)
            ans.append(y)


In [ ]:
ans

In [ ]:
sols

In [ ]:
len(coordinates)

In [ ]:
for x in coordinates:
    print(x)

In [ ]:
import numpy as np

# Define the target size
target_size = (256, 256, 3)

# List of your embeddings


# Function to pad an embedding to the target size
def pad_embedding(embedding):
    target_size = (256, 256, 3)
    padded = np.zeros(target_size)
    current_size = embedding.shape
    padded[:current_size[0], :current_size[1], :current_size[2]] = embedding
    return padded

# Pad all embeddings
padded_embeddings = [pad_embedding(e) for e in embeddings]

# Convert the list to a numpy array if needed
padded_embeddings_array = np.array(padded_embeddings)


In [ ]:
len(paths)

In [ ]:
path = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/resizedimages'
paths = os.listdir(path)
paths = [path + '/'+ x for x in paths]

known_face_encodings = generate_embeddings(paths)


In [ ]:
len(known_face_encodings)

In [ ]:
for x in known_face_encodings:
    print(x.shape)

In [ ]:
resize_ = [pad_embedding(e) for e in known_face_encodings]
resize_ = np.array(resize_)

In [ ]:
import numpy as np

import math

import math
import numpy as np



def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    similarity = dot_product / (magnitude_v1 * magnitude_v2)
    return similarity


In [ ]:
for x in padded_embeddings:
    print((x[:,:,1].shape))

In [ ]:
for y in padded_embeddings:
    y = y[:,:,::-1]
    t = y[:,:,1]
    print(t.shape)

In [ ]:
l = []
for x in padded_embeddings:
    u = []
    x = x[:, :, ::-1]
    x = x[:,:,1]
    x = x.flatten()
    for y in resize_:
        y = y[:,:,::-1]
        t = y[:,:,1]
        t = t.flatten()
        u.append(cosine_similarity(x,t))
    l.append(u)
        

In [ ]:
import numpy as np
from PIL import Image

# Load the image as a NumPy array
image = np.array(Image.open('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/expirements/Frames/frame_0.jpg'))

# Define the coordinates for the region to extract
x, y, w, h = 366, 533,48,51


# Extract the region
cropped_image = image[y:y+h, x:x+w]

# Convert back to an image and save
Image.fromarray(cropped_image).save('cropped_image.jpg')


In [ ]:
from mtcnn import MTCNN
from PIL import Image
import numpy as np

# Initialize MTCNN
detector = MTCNN()

# Load the image
image = Image.open('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/expirements/Frames/frame_1.jpg')
image_np = np.array(image)

# Detect faces
results = detector.detect_faces(image_np)

# Print the coordinates of the detected faces
for result in results:
    x, y, width, height = result['box']
    print(f'Face found at (x={x}, y={y}, width={width}, height={height})')


In [ ]:
import matplotlib.pyplot as plt
image = cv2.imread('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/expirements/Frames/frame_1.jpg')
x=180
y=585
w=100
h=126
cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

plt.imshow(image)

In [ ]:
from PIL import Image

def extract_bbox(image_path, bbox):
    # Open the image file
    with Image.open(image_path) as img:
        # bbox is a tuple (left, upper, right, lower)
        left, upper, right, lower = bbox
        
        # Crop the image to the bounding box
        cropped_image = img.crop((left, upper, right, lower))
        
        return cropped_image

# Example usage
image_path = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/expirements/Frames/frame_1.jpg'
bbox = (180, 585, 280, 711)  # Example bounding box coordinates
cropped_image = extract_bbox(image_path, bbox)
cropped_image.show()  # Display the cropped image
cropped_image.save('cropped_image.jpg')  # Save the cropped image


In [ ]:
for x in l:
    print(max(x))

In [ ]:
import cv2

# Load the image
image_path = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/expirements/Frames/frame_0.jpg'
image = cv2.imread(image_path)

# Facial area data
facial_area = {
    'x': 417,
    'y': 538,
    'w': 59,
    'h': 59
}

# Extract coordinates and dimensions
x = facial_area['x']
y = facial_area['y']
w = facial_area['w']
h = facial_area['h']

cropped_face = image[x:x+w,y:y+h]
# Save the cropped face to a file
cv2.imwrite('cropped_face.jpg', cropped_face)


## harsha's

In [ ]:
from deepface import DeepFace
import os
import csv 
def face_detections_vggface(path):
    embedding_objs = DeepFace.represent(
    img_path = path,
    model_name='VGG-Face',
    )
    face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
    return face_embeddings

def face_detections_facenet(path):
    embedding_objs = DeepFace.represent(model_name='Facenet',
        img_path = path,
        )
    face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
    return face_embeddings

def face_detections_deepface(path):
    embedding_objs = DeepFace.represent(model_name='DeepFace',
        img_path = path,
        )
    face_embeddings = [embedding_objs[x]['embedding'] for x in range(len(embedding_objs))]
    return face_embeddings
def convert_into_csv(paths,image_roll):
    vggface,facenet,deepface = [],[],[]
    for vgg_path in paths :
           vgg_encodings = face_detections_vggface(vgg_path)
           vgg_string="/".join([str(i) for i in vgg_encodings[0]])
           vggface.append(vgg_string)
    for facenet_path in paths :
         facenet_encodings = face_detections_facenet(facenet_path)
         facenet_string = "/".join([str(i) for i in facenet_encodings[0]])
         facenet.append(facenet_string)
    for deepface_path in paths:
         deepface_encodings = face_detections_deepface(deepface_path)
         deepface_string = "/".join([str(i) for i in deepface_encodings[0]])
         deepface.append(deepface_string)
    names_list = ["Sadiq","Sadikk","Sri Ram","Rohith","Porna",
                  "Dev","Bellamkkonda srinivas","Vivek boyapuda",
                  "venkat","charan Manikanta Nalla","Dinesh","Bhavani Sankar"
                ]
   
    with open("face_encodings.csv","w") as obj:
          writer = csv.writer(obj)
          writer.writerow(["Names","RollNo","vggface_encoding","facenet_encoding","deepface_encoding"])
          for i in range(len(names_list)):
               detailed_array = list(zip(names_list[i],image_roll[i].split(".")[0],vggface[i],facenet[i],deepface[i]))
               writer.writerow(detailed_array)
          


if __name__ =='__main__':
     image_paths = os.listdir('faces/')
     print(image_paths)
     full_paths = ['faces/'+i for i in image_paths]
     convert_into_csv(full_paths,image_paths)

In [ ]:
cd ..

In [ ]:
ls